In [1]:
import cv2
import numpy as np
import pickle
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
import sys
import os 
import openpyxl

ModuleNotFoundError: No module named 'keras'

In [ ]:
size = 4
datasets = 'datasets'

# Part 1: Create fisherRecognizer 
print('Recognizing Face Please Be in sufficient Lights...') 

# Create a list of images and a list of corresponding names 
(images, labels, names, id) = ([], [], {}, 0) 
for (subdirs, dirs, files) in os.walk(datasets): 
    for subdir in dirs: 
        names[id] = subdir 
        subjectpath = os.path.join(datasets, subdir) 
        for filename in os.listdir(subjectpath): 
            path = subjectpath + '/' + filename 
            label = id
            images.append(cv2.imread(path, 0)) 
            labels.append(int(label)) 
        id += 1
(width, height) = (130, 100) 

# Create a Numpy array from the two lists above 
(images, labels) = [np.array(lis) for lis in [images, labels]] 

with open('Beard_classifier.pkl', 'rb') as file:
    beard_classifier, beard_label_encoder = pickle.load(file)

def preprocess_image(img_array):
    img = cv2.resize(img_array, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def extract_feature(img_array):
    img = preprocess_image(img_array)
    feature = vgg_model.predict(img).flatten()
    return feature

# Load the workbook or create a new one
excel_file_path = 'recognized_faces.xlsx'
if os.path.exists(excel_file_path):
    workbook = openpyxl.load_workbook(excel_file_path)
    if 'New' in workbook.sheetnames:
        sheet = workbook['New']
    else:
        sheet = workbook.create_sheet('New')
else:
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = 'New'

# Function to check if a name exists in the sheet
def name_exists(sheet, name):
    for row in sheet.iter_rows(values_only=True):
        if name in row:
            return True
    return False

def clear_sheet(sheet):
    sheet.delete_rows(1, sheet.max_row)

Recognizing Face Please Be in sufficient Lights...


In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = cv2.face.LBPHFaceRecognizer_create() 
model.train(images, labels)
clear_sheet(sheet)

In [ ]:
webcam = cv2.VideoCapture(0) 
while True: 
    (_, im) = webcam.read() 
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    for (x, y, w, h) in faces: 
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2) 
        face = gray[y:y + h, x:x + w] 
        face_crop = im[y:y+h, x:x+w]
        face_resize = cv2.resize(face, (width, height)) 
        feature = extract_feature(face_crop)
        # Try to recognize the face 
        prediction = model.predict(face_resize)
        beard_prediction = beard_classifier.predict([feature])
        beard_name = beard_label_encoder.inverse_transform(beard_prediction)[0]
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3) 

        if prediction[1]<500:
                        name = '% s' % (names[prediction[0]])
                        cv2.putText(im,  name+ beard_name, (x-10, y-10),  cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
                        # Add name to Excel sheet if it doesn't exist
                        if not name_exists(sheet, name):
                            sheet.append([name,beard_name])
                            workbook.save(excel_file_path)
                            print(f"Added '{name}' to the Excel sheet.")
        else:
                        cv2.putText(im, 'not recognized', (x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0)) 

    cv2.imshow('OpenCV', im) 
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
Added 'Yuvaraj' to the Excel sheet.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
